## Assignment 4
### Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [wikipedia_data.html](wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [nfl.csv](nfl.csv)), MLB (baseball, in [mlb.csv](mlb.csv)), NBA (basketball, in [nba.csv](nba.csv) or NHL (hockey, in [nhl.csv](nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

### Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [187]:
import pandas as pd
import numpy as np
import scipy.stats as stats

def nhl_correlation():
        nhl_df = pd.read_csv('nhl.csv')
        nhl_df = nhl_df[nhl_df['year'] == 2018]
        nhl_df = nhl_df[['team','W', 'L']]
        nhl_df.columns = ['team_only', 'W', 'L']
        nhl_df = nhl_df[~nhl_df['W'].str.contains('Division')]
        nhl_df['team_only'] = nhl_df['team_only'].str.split('*').str[0]
        nhl_df['team_only'] = nhl_df['team_only'].str.split(' ').str[-1]
        
        nhl_df['W'] = nhl_df['W'].astype(float)
        nhl_df['L'] = nhl_df['L'].astype(float)
        
        nhl_df.loc[3, 'team_only'] = 'Maple Leafs'
        nhl_df.loc[5, 'team_only'] = 'Red Wings'
        nhl_df.loc[13,'team_only'] = 'Blue Jackets'
        nhl_df.loc[27,'team_only'] = 'Golden Knights'
        
        nhl_df.loc[14,'team_only'] = 'Rangers Islanders Devils'
        nhl_df.loc[16,'team_only'] = 'Rangers Islanders Devils'
        nhl_df.loc[17,'team_only'] = 'Rangers Islanders Devils'
        nhl_df.loc[28,'team_only'] = 'Kings Ducks'
        nhl_df.loc[30,'team_only'] = 'Kings Ducks'

        nhl_df = nhl_df.groupby('team_only')[['W','L']].mean()
        cities = pd.read_html('wikipedia_data.html')[1]
        cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
        cities.columns = ['cities', 'population', 'team_only']
        cities['team_only'] = cities['team_only'].str.split('[').str[0]
        
        df = pd.merge(nhl_df, cities, how='inner', on='team_only')
        df['win_loss_ratio'] = df['W']/(df['W'] + df['L'])
        
        population_by_region = pd.to_numeric(df['population'], errors='coerce')
        win_loss_by_region = pd.to_numeric(df['win_loss_ratio'], errors='coerce')
        
        return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012308996455744266

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [270]:
def nba_correlation():
        nba_df = pd.read_csv('nba.csv')
        nba_df = nba_df[nba_df['year'] == 2018]
        nba_df = nba_df[['team', 'W', 'L']]
        nba_df['team'] = nba_df['team'].str.split('*').str[0]
        nba_df['team'] = nba_df['team'].str.split('(').str[0]
        nba_df['team'] = nba_df['team'].str.split().str[-1]
        nba_df.columns = ['team_only', 'W','L']
        nba_df['W'] = nba_df['W'].astype(float)
        nba_df['L'] = nba_df['L'].astype(float)
        
        nba_df.loc[10, 'team_only'] = 'Knicks Nets'
        nba_df.loc[11, 'team_only'] = 'Knicks Nets'
        nba_df.loc[24, 'team_only'] = 'Lakers Clippers'
        nba_df.loc[25, 'team_only'] = 'Lakers Clippers'
        nba_df.loc[17, 'team_only'] = 'Trail Blazers'
        
        nba_df = nba_df.groupby('team_only')[['W', 'L']].mean()
        
        cities = pd.read_html('wikipedia_data.html')[1]
        cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
        cities['NBA'] = cities['NBA'].str.split('[').str[0]
        cities.columns = ['city', 'population', 'team_only']
        
        merged_df = pd.merge(nba_df, cities, how='inner', on='team_only')
        merged_df['win_loss_ratio'] = merged_df['W']/(merged_df['W'] + merged_df['L'])
        population_by_region = pd.to_numeric(merged_df['population'], errors='coerce')
        win_loss_by_region = pd.to_numeric(merged_df['win_loss_ratio'], errors='coerce')
        
        return population_by_region.corr(win_loss_by_region)

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [314]:
def mlb_correlation():
        mlb_df = pd.read_csv('mlb.csv')
        mlb_df = mlb_df[mlb_df['year'] == 2018]
        mlb_df = mlb_df[['team', 'W', 'L']]
        mlb_df['team'] = mlb_df['team'].str.split().str[-1]
        mlb_df.columns = ['team_only', 'W', 'L']
        mlb_df['W'] = mlb_df['W'].astype(float)
        mlb_df['L'] = mlb_df['L'].astype(float)
        
        mlb_df.loc[0, 'team_only'] = 'Red Sox'
        mlb_df.loc[1, 'team_only'] = 'Yankees Mets'
        mlb_df.loc[18,'team_only'] = 'Yankees Mets'
        mlb_df.loc[13,'team_only'] = 'Dodgers Angels'
        mlb_df.loc[25,'team_only'] = 'Dodgers Angels'
        mlb_df.loc[11,'team_only'] = 'Giants Athletics'
        mlb_df.loc[28,'team_only'] = 'Giants Athletics'
        mlb_df.loc[8, 'team_only'] = 'Cubs White Sox'
        mlb_df.loc[21,'team_only'] = 'Cubs White Sox'
        mlb_df.loc[3, 'team_only'] = 'Blue Jays'
        
        mlb_df = mlb_df.groupby('team_only')[['W', 'L']].mean()
        
        cities = pd.read_html('wikipedia_data.html')[1]
        cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
        cities['MLB'] = cities['MLB'].str.split('[').str[0]
        cities.columns = ['city', 'population', 'team_only']
        
        
        merged_df = pd.merge(mlb_df, cities, how='inner', on='team_only')
        merged_df['win_loss_ratio'] = merged_df['W']/(merged_df['W'] + merged_df['L'])
        population_by_region = pd.to_numeric(merged_df['population'], errors='coerce')
        win_loss_by_region = pd.to_numeric(merged_df['win_loss_ratio'], errors='coerce')
        
        return population_by_region.corr(win_loss_by_region)

mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [348]:
def nfl_correlation():
        nfl_df = pd.read_csv('nfl.csv')
        nfl_df = nfl_df[nfl_df['year'] == 2018]
        nfl_df = nfl_df[['team', 'W', 'L']]
        nfl_df = nfl_df[~nfl_df['W'].str.contains('AFC')]
        nfl_df = nfl_df[~nfl_df['W'].str.contains('NFC')]
        nfl_df = nfl_df.reset_index(drop=True)
        
        nfl_df['team'] = nfl_df['team'].str.split('*').str[0]
        nfl_df['team'] = nfl_df['team'].str.split('+').str[0]
        nfl_df['team'] = nfl_df['team'].str.split().str[-1]
        nfl_df.columns = ['team_only', 'W', 'L']
        nfl_df['W'] = nfl_df['W'].astype(float)
        nfl_df['L'] = nfl_df['L'].astype(float)
        
        nfl_df.loc[3, 'team_only'] = 'Giants Jets'
        nfl_df.loc[19,'team_only'] = 'Giants Jets'
        nfl_df.loc[13,'team_only'] = 'Rams Chargers'
        nfl_df.loc[28,'team_only'] = 'Rams Chargers'
        nfl_df.loc[15,'team_only'] = '49ers Raiders'
        nfl_df.loc[30,'team_only'] = '49ers Raiders'
        
        nfl_df = nfl_df.groupby('team_only')[['W', 'L']].mean()
        
        cities = pd.read_html('wikipedia_data.html')[1]
        cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
        cities['NFL'] = cities['NFL'].str.split('[').str[0]
        cities.columns = ['city', 'population', 'team_only']
        
        merged_df = pd.merge(nfl_df, cities, how='inner', on='team_only')
        merged_df['win_loss_ratio'] = merged_df['W']/(merged_df['W'] + merged_df['L'])
        population_by_region = pd.to_numeric(merged_df['population'], errors='coerce')
        win_loss_by_region = pd.to_numeric(merged_df['win_loss_ratio'], errors='coerce')
        
        return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349415

### Thank you
## Author

[Neelesh Chidar](https://www.linkedin.com/in/neeleshchidar37/)